In [80]:
!pip install openai anthropic IPython

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display
import time

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
# anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
# deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

Defaulting to user installation because normal site-packages is not writeable


In [81]:
# Checking keys
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
# if anthropic_api_key:
#     print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
# else:
#     print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

# if deepseek_api_key:
#     print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
# else:
#     print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
Groq API Key exists and begins gsk_


In [84]:
# Making a Question!
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Make output in markdown format design"
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

# Setup question and display with markdown + timer
openai = OpenAI()

q_start_time = time.time() 
response = openai.chat.completions.create(
    model="gpt-5-nano",
    messages=messages,
)
question = response.choices[0].message.content

q_end_time = time.time()
duration = q_end_time - q_start_time
display(Markdown(question))
print(f"\nTime taken for API call for question: **{duration:.4f} seconds**")

```markdown
Design a robust decision framework for selecting a single policy under extreme uncertainty in a partially observable environment with noisy and potentially adversarial data. Your answer should include:
1) A model-agnostic decision rule or algorithm that is implementable with limited data and minimizes overfitting while providing interpretable rationale for the chosen policy.
2) A clear set of assumptions, data requirements, and a plan to empirically test these assumptions using hypothetical but plausible data scenarios.
3) At least three falsifiable predictions about outcomes under three distinct regimes (e.g., regime A: stable data-generating process with modest noise; regime B: non-stationarity and concept drift; regime C: adversarial manipulation).
4) A monitoring and update protocol to detect distributional shift, data integrity issues, and biases, including trigger conditions, escalation paths, and rollback procedures.
5) A thorough assessment of ethical, legal, and social implications for stakeholders, including potential biases and concrete mitigation strategies.
6) A concise justification for why the approach is likely to perform well in practice, plus a discussion of key limitations and scenarios where it may fail.
Please provide the final decision rule and justification in a structured, implementable format and avoid exposing step-by-step internal chain-of-thought.
```


Time taken for API call for question: **26.0937 seconds**


In [86]:
# Setup for the competition!

competitors = []
answers = []
durations = []
question += "Don't make too long answer and arrange the output in Markdown format. No continuation of the chat, stop after the answer."
messages = [{"role": "user", "content": question}]

In [88]:
# Competitor1

model_name = "gpt-5-mini"

c1_start_time = time.time()

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

c1_end_time = time.time()
c1_duration = c1_end_time - c1_start_time

print(f"\nTime taken for {model_name} API call: **{c1_duration:.4f} seconds**")
display(Markdown(answer))

competitors.append(model_name)
answers.append(answer)
durations.append(c1_duration)


Time taken for gpt-5-mini API call: **63.0068 seconds**


# Robust Decision Framework for Selecting a Single Policy under Extreme Uncertainty

Summary: a model-agnostic, implementable rule (Robust Lower-bound Selection, RLS) that (a) evaluates candidate policies under an ensemble of plausible perturbations (noise, shifts, adversarial manipulations), (b) chooses the policy that maximizes a conservative performance metric (worst-case lower confidence bound or CVaR) while penalizing complexity, (c) produces an interpretable rationale and monitoring/rollback plan.

---

## 1. Final decision rule (algorithm: Robust Lower-bound Selection — RLS)

Inputs
- Candidate policy set P = {π1,...,πk} (can be rules, parametric policies, or black-box policies).
- History dataset D = {(x_i, a_i, r_i, t_i, meta_i)} (covariates x, action a, reward/outcome r, timestamp t, metadata).
- Baseline policy π0 (safe fallback).
- Utility function U(π; D) (e.g., expected reward, harm-weighted utility).
- Uncertainty generators G = {g1,...,gm} describing plausible perturbations (noise levels, covariate shifts, label corruption, adversarial fraction ε).
- Off-policy evaluator E (IPS with clipped weights or Doubly Robust), bootstrap count B, confidence level α (e.g., 0.05), CVaR tail τ (optional, e.g., 0.1).
- Complexity penalty C(π) (e.g., model size, interpretability cost), weight λ.
- Trigger thresholds for monitoring (see §4).

Outputs
- Selected policy π* and interpretability report R (LCBs/CVaR across scenarios, sensitivity summary, failure modes).
- Deployment plan & rollback steps.

Procedure
1. Data sanitization:
   - Basic integrity checks (missingness, duplicates, signature/hashes), remove/flag corrupted rows.
   - Estimate propensity scores p(a|x) if needed for IPS; apply clipping at w_max.

2. Build perturbation scenarios:
   - For each g ∈ G, define parameterized perturbations (e.g., additive noise σ, covariate shift magnitude δ, label flip rate η, fraction ε of adversarial examples).
   - Use domain knowledge to bound ranges (conservative priors).

3. Bootstrap evaluation:
   - For b = 1..B:
     - Resample D_b from D (with stratification if needed).
     - For each scenario s generated from G:
       - Apply perturbation s to D_b → D_{b,s}.
       - For each candidate policy π:
         - Compute off-policy estimate U_{b,s}(π) using E on D_{b,s}; record estimator variance.
   - This yields distribution of U across (b,s) for each π.

4. Compute robust score per π:
   - Option A (Lower-confidence bound): LCB(π) = empirical α-quantile of U_{b,s}(π) across all (b,s).
   - Option B (CVaR): CVaR_τ(π) = mean of worst τ fraction of U_{b,s}(π).
   - RobustScore(π) = LCB(π) − λ·C(π) (or CVaR_τ(π) − λ·C(π)).
   - Optionally compute worst-case regret relative to baseline: WorstRegret(π) = max_{b,s} (U_{b,s}(π0) − U_{b,s}(π)).

5. Selection:
   - Choose π* = argmax_π RobustScore(π), subject to constraint WorstRegret(π) ≤ R_max (set R_max per risk tolerance). If no π meets constraints, default to baseline π0.

6. Rationale and interpretability output R:
   - Report LCB/CVaR and median for chosen π*, complexity cost, most damaging perturbation scenarios, sensitivity charts, and policy decision rule summary.
   - Provide simple explanation: e.g., "π* selected because its 95% lower bound on expected utility under plausible perturbations is greatest and worst-case regret vs baseline ≤ X."

7. Deploy with guardrails:
   - Deploy π* in staged rollout (canary, A/B with heavy monitoring), see §4 for triggers and rollback.

Implementation notes
- Off-policy evaluation: use Doubly Robust or weighted IPS; clip importance weights and report sensitivity to clipping.
- Keep candidate policies simple where possible (rules, small trees, linear policies) to improve interpretability and reduce overfitting.
- Choose B (e.g., 500) and number of scenarios m pragmatically—enough for stable estimates given data size.

---

## 2. Assumptions, data requirements, and empirical test plan

Assumptions (explicit)
- A1: Utility function U correctly captures stakeholder objectives (or a defensible proxy exists).
- A2: Candidate policies P contain at least one safe fallback π0.
- A3: Historical data D is informative for the short-term behavior of the system (coverage of relevant covariate-action space).
- A4: Perturbation family G contains realistic bounds for noise, shift, and adversary strength (conservative, but plausible).
- A5: Off-policy evaluation bias is tolerable with clipping/DR corrections (no extreme propensities).

Data requirements (minimum)
- N_total: at least a few hundred independent episodes per key stratum; if scarce, rely more on conservative G and simpler policies.
- Metadata: timestamps, provenance, and integrity checks.
- Feature coverage: key covariates that influence action-outcome relationships must appear in D.
- Baseline behavior data for π0 to compute regret.

Plan to empirically test assumptions (hypothetical scenarios)
- Create synthetic datasets that mimic domain features and labeled outcomes; vary noise, shift, adversarial fractions.
- Tests:
  - Coverage test (A3): remove certain covariate strata from training data; measure IPS variance and LCB widening.
  - Sensitivity to G (A4): expand/shrink perturbation ranges; confirm selection stability (π* stable under reasonable changes).
  - OPE validity (A5): compare off-policy estimates to on-policy returns in small controlled online rollouts (canary) to validate E calibration.
- Acceptance criteria:
  - If empirical LCBs consistently underestimate true on-policy canary returns by >δ, recalibrate E or tighten rolling deployment.
  - If candidate selection flips across many plausible G changes, increase conservatism (larger λ or stricter WorstRegret).

---

## 3. Falsifiable predictions (three regimes)

Define metric M = realized average utility per episode measured on holdout/on-policy canary.

Regime A — Stable, modest noise
- Prediction A1: Within T_A (e.g., 2 weeks or 10k episodes), M(π*) ≥ median LCB(π*) + ε_A (ε_A small, e.g., 0.02 utility units).
- Falsifiability: if M(π*) < LCB(π*) − δ_A, prediction fails.

Regime B — Non-stationarity/concept drift
- Prediction B1: M(π*) will degrade gradually; degradation slope s satisfies |ΔM/Δt| ≤ s_max over initial window W (we expect detect-and-update triggers before performance drops below baseline).
- Falsifiability: if M(π*) drops abruptly below baseline π0 without preceding drift signals (distributional detectors not triggered), prediction fails.

Regime C — Adversarial manipulation
- Prediction C1: Under adversarial fraction ≤ ε_adversary (modeled in G), worst-case observed M(π*) ≥ WorstCase_LCB(π*) − ε_C (conservative), and monitoring will detect integrity anomalies (metadata tampering, abnormal reward patterns) within detection window T_C.
- Falsifiability: if adversary-induced outcomes produce M(π*) significantly below WorstCase_LCB and no integrity flags are raised in T_C, prediction fails.

Each prediction is falsifiable by comparing on-policy logged metrics to predicted intervals and by inspecting detector triggers.

---

## 4. Monitoring and update protocol (detection, escalation, rollback)

Continuous monitoring components
- Data-distribution monitors:
  - Covariate shift: population stability index (PSI) and KL divergence on key features; threshold θ_cov (e.g., PSI > 0.2).
  - Label/Outcome shift: compare recent outcome distribution to baseline with Earth Mover’s Distance; threshold θ_lab.
- Model/performance monitors:
  - Utility drift: rolling average of utility M with statistical control charts (CUSUM/Shewhart) at significance α_mon.
  - Off-policy vs on-policy divergence: compare OPE predictions to canary actuals.
- Integrity/anomaly monitors:
  - Metadata validation (signatures, device IDs), outlier detection on timestamps and event rates.
  - Adversarial detectors: sudden cluster of contradictory examples, reward spikes, or disproportionate error rates.
- Fairness/bias monitors:
  - Disparate impact (ratio metrics), subgroup utilities, intersectional checks; thresholds set by policy.

Trigger conditions (examples)
- Tier 1 (warning): PSI > θ1 or utility drift p-value < 0.05 → notify operations and data science.
- Tier 2 (action required): PSI > θ2 or utility drop > X% vs baseline OR integrity anomaly → pause new exposures to π*; initiate canary rollback to π0 for affected segments.
- Tier 3 (urgent rollback): evidence of data tampering or targeted harm (meet legal/ethical stop criteria) → immediate full rollback to π0 and incident response.

Escalation path
- Tier 1: Automated alert → data science on-call inspects dashboards within 4 hours.
- Tier 2: Convene cross-functional response (ops, DS, legal, compliance) within 2 hours; restrict traffic to canary; collect forensic logs.
- Tier 3: Immediate cessation; notify leadership and regulators as required; start post-incident review.

Rollback & recovery procedures
- Pre-deploy: ensure π0 is always deployable, versioned, and tested; keep feature transforms consistent.
- Immediate rollback: switch routing to π0; preserve logs; mark affected data for quarantine.
- Post-rollback: root-cause analysis, patch G and monitoring thresholds, retrain or replace candidate set if needed.
- Audit trail: full logging of decision, metrics, time of rollback, and rationale; share with stakeholders.

---

## 5. Ethical, legal, and social implications & mitigation

Key stakeholder impacts
- Users potentially harmed by mistaken decisions (safety-critical domains).
- Subpopulations disproportionately affected (bias).
- Data subjects’ privacy and consent.
- Organizational/legal exposure (regulation compliance).

Risks
- Distributional bias causing disproportionate harms.
- Adversarial exploitation leading to targeted harms.
- Over-reliance on automated decision with insufficient human oversight.
- Privacy leaks from logs or models.

Concrete mitigation strategies
- Fairness constraints: enforce minimum subgroup utility floors during selection (add constraints to WorstRegret or per-group LCB≥g_min).
- Transparency & explanation: produce R reporting interpretable rationale for chosen policy; publish summary for stakeholders.
- Human-in-the-loop: require human approval for high-risk deployments and for Tier 2+ escalations.
- Data governance: strict provenance, access controls, hashing/signatures for integrity, and retention policies.
- Privacy protections: minimize sensitive attributes in decisioning; apply differential privacy or noise to analytics when needed.
- Adversarial hardening: red-team tests, simulated adversary scenarios in G, input sanity checks.
- Legal compliance: consult counsel for sector rules (e.g., GDPR, HIPAA); implement opt-out and data subject rights flows.
- Independent audits: periodic external fairness/safety audits and public reporting where appropriate.

---

## 6. Justification, expected performance, and limitations

Why this will perform well in practice
- Conservative selection (LCB/CVaR) reduces chance of worst-case catastrophic outcomes under uncertainty—suitable when stakes are high.
- Model-agnostic: works with rules, simple models, or complex policies; off-policy evaluation avoids extensive online exploration.
- Bootstrapping + scenario perturbations produce robust uncertainty quantification even with limited data.
- Complexity penalty and preference for simpler policies reduce overfitting and improve interpretability.

Key limitations and failure modes
- Conservatism: may select suboptimal policy for benign environments (opportunity cost).
- Unmodeled "unknown unknowns": if real-world perturbations lie outside G, selection may fail.
- Dependence on OPE accuracy: heavy bias in E (poor propensities, extreme covariate mismatch) undermines estimates.
- Candidate-policy coverage: if P lacks any robust good policy, selection cannot fix it—requires human design loop.
- Data scarcity: with very small N, LCBs are wide and decision may default to baseline frequently.
- Sophisticated adversary: if adversary can adapt in deployment beyond modeled capacity, monitoring might lag.

Mitigations for limitations
- Periodic expansion of G via adversarial red-teaming; incorporate rapid canary cycles to validate OPE.
- Maintain policy development pipeline to generate new candidates when selection repeatedly returns π0.
- Combine with conservative online learning only under strict safety constraints.

---

## Quick parameter guidance (practical defaults)
- B (bootstrap): 200–1000 depending on compute.
- α for LCB: 0.05 (95% lower bound).
- CVaR τ: 0.1 if risk-averse.
- Importance weight clipping w_max: 10.
- Complexity penalty λ: tuned so simple policy preferred unless complex policy improves LCB by >δ.
- Monitoring windows: short-term (hourly) for integrity; medium-term (daily) for utility drift.

---

## Deliverables produced by the framework
- Selected policy π* with decision report R (LCBs, worst-case scenarios, sensitivity).
- Monitoring dashboard with PSI, utility charts, fairness metrics, integrity logs.
- Escalation playbook and rollback scripts (automated routing to π0).
- Test suite of simulated scenarios used to validate assumptions.

---

This framework is concise, implementable with limited data, and designed to minimize overfitting while producing interpretable justification for the chosen policy. It trades some short-term performance for robustness and safety under extreme uncertainty; use domain expertise to set G, thresholds, and the baseline policy.

In [90]:
# Anthropic has a slightly different API, and Max Tokens is required

# model_name = "claude-sonnet-4-5"

# claude = Anthropic()
# response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
# answer = response.content[0].text

# display(Markdown(answer))
# competitors.append(model_name)
# answers.append(answer)


# Deepseek
# deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
# model_name = "deepseek-chat"

# response = deepseek.chat.completions.create(model=model_name, messages=messages)
# answer = response.choices[0].message.content

# display(Markdown(answer))
# competitors.append(model_name)
# answers.append(answer)

In [92]:
# Competitor 2

gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"

c2_start_time = time.time()

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

c2_end_time = time.time()
c2_duration = c2_end_time - c2_start_time

print(f"Time taken for {model_name}: **{c2_duration:.4f} seconds**\n")
display(Markdown(answer))

competitors.append(model_name)
answers.append(answer)
durations.append(c2_duration)

Time taken for gemini-2.5-flash: **39.5234 seconds**



The following robust decision framework is designed for selecting a single policy under extreme uncertainty, partial observability, and with noisy/adversarial data.

---

### **1. Model-Agnostic Decision Rule: Robust Scenario-Based Policy Selection (RSBPS)**

The RSBPS framework prioritizes robustness and interpretability over predictive accuracy, which is highly challenging under extreme uncertainty and adversarial conditions. It identifies a policy that minimizes the maximum regret across a diverse set of plausible, stressful, and adversarial future scenarios.

**Input:**
*   A finite set of candidate policies: $P = \{p_1, p_2, \dots, p_k\}$.
*   A set of critical uncertainties and features derived from the partially observable, noisy data: $U = \{u_1, u_2, \dots, u_n\}$. These represent key variables or processes that significantly impact policy outcomes but are subject to high uncertainty or manipulation.
*   A performance metric: $M(p_i, s_j)$, a scalar value representing the outcome (e.g., cost, safety, efficiency, impact) of applying policy $p_i$ under scenario $s_j$. Assume $M$ is defined such that *lower values are better* (e.g., lower cost, fewer failures). If higher values are better, adjust regret calculation accordingly.

**Algorithm:**

1.  **Scenario Construction (S):**
    *   **Define Plausible & Adversarial Scenarios:** Based on current and historical data, expert knowledge, and known adversarial tactics, construct a comprehensive set of future scenarios $S = \{s_1, s_2, \dots, s_m\}$. Each $s_j$ describes a specific realization of the critical uncertainties $U$.
    *   **Types of Scenarios:**
        *   **Nominal:** Most likely or expected conditions (e.g., stable data-generating process with modest noise).
        *   **Stress-Test:** Extreme but plausible conditions (e.g., significant but random data noise, unexpected market shifts, resource scarcity).
        *   **Adversarial:** Conditions where data is actively manipulated or system behavior is targeted (e.g., data poisoning, spoofing, denial-of-service, concept drift induced by an adversary).
        *   **Drift/Non-stationarity:** Scenarios explicitly modeling shifts in data distributions or underlying processes.
        *   **Black Swan (if conceivable):** Highly improbable but high-impact events.
    *   *Rationale:* This step integrates domain expertise and adversarial thinking directly into the decision process, bypassing the need for complex predictive models.

2.  **Policy Performance Evaluation:**
    *   For each policy $p_i \in P$ and each scenario $s_j \in S$, estimate or simulate the performance metric $M(p_i, s_j)$. This step may involve simplified models, expert elicitation, or historical data-driven simulations tailored to each scenario's conditions.
    *   *Rationale:* Directly assesses policy outcomes under diverse conditions, rather than relying on a single, potentially flawed model of the environment.

3.  **Regret Calculation:**
    *   For each scenario $s_j$, identify the *optimal* performance among all candidate policies: $M_{optimal}(s_j) = \min_{p_i \in P} \{M(p_i, s_j)\}$.
    *   Calculate the *scenario-specific regret* for each policy $p_i$ under scenario $s_j$: $R(p_i, s_j) = M(p_i, s_j) - M_{optimal}(s_j)$. This measures how much worse policy $p_i$ performs compared to the best possible policy *for that specific scenario*.
    *   *Rationale:* Focuses on relative performance, which is more robust than absolute performance in highly uncertain environments.

4.  **Max Regret Identification:**
    *   For each policy $p_i$, identify its *maximum regret* across all scenarios: $MaxRegret(p_i) = \max_{s_j \in S} \{R(p_i, s_j)\}$. This represents the worst-case outcome for policy $p_i$ relative to the optimal choice for any given scenario.
    *   *Rationale:* Explicitly identifies and quantifies the "least worst" performance, preparing for pessimistic outcomes.

5.  **Policy Selection (Minimax Regret Criterion):**
    *   Select the policy $p^*$ that minimizes the maximum regret:
        $p^* = \arg \min_{p_i \in P} \{MaxRegret(p_i)\}$
    *   *Rationale:* This "minimax regret" strategy selects the policy that guarantees the best possible outcome under the worst possible scenario (relative to the optimal choice for that scenario). It's a robust choice when facing deep uncertainty and potential adversaries.

**Interpretable Rationale:** The selected policy $p^*$ is chosen because it is the most robust option, minimizing the maximum potential "missed opportunity" or "sub-optimality" across all considered scenarios. Its rationale is directly tied to its comparative performance across the explicitly defined plausible, stressful, and adversarial futures.

---

### **2. Assumptions, Data Requirements, and Empirical Testing Plan**

**Assumptions:**
*   **Existence of Candidate Policies:** A finite, well-defined set of policies exists for consideration.
*   **Feasibility of Scenario Definition:** It's possible to define a sufficiently rich and relevant set of scenarios (including adversarial ones) that capture the critical uncertainties.
*   **Measurable Performance:** A quantifiable, agreed-upon performance metric $M$ can be defined for any policy under any scenario.
*   **Bounded Rationality of Adversaries (for adversarial scenarios):** While adversaries are malicious, their capabilities and objectives are not entirely unknowable; there are limits to their manipulation.
*   **Observable Indicators (even if noisy):** Despite partial observability and noise, some indicators or features (e.g., changes in correlation, distribution shifts, specific attack signatures) can be detected to inform scenario updates and monitoring.

**Data Requirements:**
*   **Historical Policy Performance Data:** Records of how past policies performed under varying conditions (if available).
*   **Feature Data:** Current and historical data streams relevant to the critical uncertainties $U$, even if noisy or sparse. This informs scenario definition and monitoring.
*   **Expert Elicitation:** Input from domain experts on plausible scenarios, potential adversarial tactics, and policy outcomes in specific situations.
*   **Adversarial Intelligence:** Data on known attack vectors, adversarial capabilities, and objectives from security reports or threat intelligence.
*   **Policy Specifications:** Detailed descriptions of each candidate policy $p_i$.

**Empirical Testing Plan (Hypothetical Scenarios):**

1.  **Regime A: Stable Data-Generating Process with Modest Noise**
    *   **Hypothetical Setup:** Simulate a stable environment where underlying processes are constant, and data noise follows a well-behaved distribution (e.g., Gaussian). Candidate policies are designed for this environment.
    *   **Test:**
        *   Define scenarios including the nominal stable state and slight variations due to noise.
        *   Run RSBPS to select a policy.
        *   Generate multiple independent data runs from the stable process.
        *   Evaluate the selected policy's performance.
    *   **Expected Outcome:** RSBPS should consistently select a policy close to the empirically optimal policy for the stable regime, demonstrating its ability to find a robust choice even when uncertainty is minimal.

2.  **Regime B: Non-stationarity and Concept Drift**
    *   **Hypothetical Setup:** Introduce a gradual or abrupt shift in the data-generating process (e.g., changing correlations between features, altered underlying distributions) after an initial stable period.
    *   **Test:**
        *   Initially, use RSBPS under the stable regime.
        *   Introduce the concept drift.
        *   Use the monitoring protocol (Section 4) to detect the shift.
        *   Upon detection, re-run RSBPS with *updated scenarios* that incorporate the detected drift.
        *   Compare the performance of the *initial* selected policy versus the *re-selected* policy after drift detection.
    *   **Expected Outcome:** The monitoring system should reliably detect the drift. RSBPS, when re-run with updated scenarios, should select a different, more appropriate policy that performs significantly better in the drifted environment than the initial policy.

3.  **Regime C: Adversarial Manipulation**
    *   **Hypothetical Setup:** Simulate an environment where a subset of input data is maliciously manipulated (e.g., data poisoning, feature injection) by an adversary aiming to degrade system performance or steer policy choice.
    *   **Test:**
        *   Define adversarial scenarios within RSBPS, explicitly modeling the adversary's capabilities and goals.
        *   Run RSBPS.
        *   Simulate the adversarial attacks on the data streams.
        *   Evaluate the selected policy's performance under these attacks, and also evaluate other "less robust" policies (e.g., one selected by a purely predictive model without adversarial scenarios).
    *   **Expected Outcome:** The policy selected by RSBPS should demonstrate greater resilience (i.e., less degradation in performance) against the simulated adversarial attacks compared to policies not chosen with adversarial scenarios in mind. Monitoring should detect integrity issues.

---

### **3. Falsifiable Predictions**

1.  **Regime A (Stable Data-Generating Process with Modest Noise):**
    *   **Prediction:** If the data-generating process is stable with modest noise, the RSBPS framework will select a policy whose average performance over 100 independent trials falls within 5% of the empirically optimal policy's average performance, and the selected policy will remain unchanged for at least 90% of re-evaluations (if no monitoring triggers).
    *   **Falsification:** If the chosen policy's average performance is consistently more than 5% worse than the empirically optimal, or if the policy selection frequently changes without external triggers, the prediction is falsified.

2.  **Regime B (Non-stationarity and Concept Drift):**
    *   **Prediction:** When a significant concept drift (defined as a statistical distribution shift detectable by a Kolmogorov-Smirnov test at p<0.01 between pre- and post-drift data windows) occurs, the monitoring protocol will trigger a re-evaluation within 3 data cycles, leading to RSBPS selecting a *different* policy from the original one in at least 80% of drift events, and this new policy will perform at least 10% better than the original policy would have in the post-drift environment.
    *   **Falsification:** If drift is not detected, or if the selected policy does not change, or if the new policy does not significantly outperform the old one post-drift, the prediction is falsified.

3.  **Regime C (Adversarial Manipulation):**
    *   **Prediction:** If 10% of input data is subjected to known adversarial poisoning attacks (e.g., label flipping, specific feature manipulation), the policy selected by RSBPS (which explicitly considers adversarial scenarios) will experience at most a 15% degradation in its performance metric, whereas a policy selected without considering adversarial scenarios would experience a degradation exceeding 30%. The monitoring system will also flag data integrity issues in over 95% of attack instances.
    *   **Falsification:** If the RSBPS-selected policy degrades by more than 15%, or if the non-adversarially chosen policy degrades by less than 30%, or if the monitoring system fails to detect most attacks, the prediction is falsified.

---

### **4. Monitoring and Update Protocol**

This protocol ensures the continued relevance and safety of the selected policy.

**Key Components:**
*   **Data Quality & Integrity Monitors:**
    *   **Scope:** Raw input data, processed features, intermediary model outputs (if any).
    *   **Checks:** Value range, completeness, consistency, statistical anomalies (e.g., sudden spikes/drops, outliers), feature correlations, entropy levels.
    *   **Techniques:** Control charts (e.g., CUSUM, EWMA), statistical tests (e.g., Z-test for means, Chi-squared for distributions), anomaly detection algorithms (e.g., Isolation Forest, Autoencoders).
*   **Distributional Shift Monitors:**
    *   **Scope:** Key feature distributions, target variable distribution (if applicable), relationship between features and outcomes.
    *   **Checks:** Change in mean, variance, skewness, kurtosis; shifts in multivariate distributions.
    *   **Techniques:** Drift detection algorithms (e.g., ADWIN, DDM), statistical tests (e.g., Kolmogorov-Smirnov, Jensen-Shannon divergence, Adversarial validation), concept drift detectors.
*   **Performance Monitors:**
    *   **Scope:** Actual observed policy performance against the chosen metric $M$.
    *   **Checks:** Deviations from expected performance, increasing variance, unexpected outcomes.
    *   **Techniques:** A/B testing (if multiple policies are run in parallel in sub-segments), backtesting against historical data, control charts on performance metrics.
*   **Bias Monitors:**
    *   **Scope:** Disparate impact across defined demographic groups or critical sub-populations, fairness metrics (e.g., Equal Opportunity, Demographic Parity).
    *   **Checks:** Discrepancies in performance $M$ or other impact metrics across groups.
    *   **Techniques:** Regular auditing of group-specific outcomes.

**Trigger Conditions:**
*   **Severity 1 (Warning):**
    *   Minor statistical anomalies or deviations (e.g., 1-sigma deviation in a control chart).
    *   Small, localized shifts in feature distributions that don't yet impact overall performance.
    *   Slight increase in uncertainty estimates.
    *   *Action:* Automated alert to monitoring team, increased logging, deeper automated diagnostics.
*   **Severity 2 (Alert):**
    *   Persistent or significant statistical anomalies (e.g., 2-sigma deviation).
    *   Detected distributional shift (e.g., K-S test p-value < 0.05).
    *   Observed performance degradation (e.g., $M$ deviates by 5-10% from expected).
    *   *Action:* Immediate alert to human operators/experts, detailed diagnostic report, initiation of a review process.
*   **Severity 3 (Critical):**
    *   Catastrophic data integrity issues (e.g., data source offline, massive corruption).
    *   Major concept drift or non-stationarity (e.g., K-S test p-value < 0.01, ADWIN/DDM triggers).
    *   Significant performance degradation (e.g., $M$ deviates by >10% from expected, or violates safety thresholds).
    *   Detection of known adversarial attack signatures.
    *   *Action:* Emergency protocol initiation, immediate policy review, potential temporary policy rollback.

**Escalation Paths:**
1.  **Automated Anomaly Detection (Severity 1):** System generates internal alerts and logs.
2.  **Human Monitoring Team (Severity 2):** Designated team investigates alerts, performs root cause analysis, and determines if a policy re-evaluation is needed.
3.  **Expert Panel/Stakeholder Review (Severity 3):** If critical issues arise, or a policy change is proposed, a panel of domain experts and key stakeholders (including ethical review board members) convenes to review diagnostics, assess new scenarios, and approve policy re-selection or emergency measures. This includes reviewing potential new adversarial scenarios.
4.  **Policy Re-evaluation:** If drift or adversarial activity is confirmed, the RSBPS framework is re-run with updated scenarios reflecting the new reality, potentially leading to a new policy selection.

**Rollback Procedures:**
*   **Temporary Rollback:** In a critical situation, revert to a previously known "safe" or default policy. This policy should be pre-approved as a stable, albeit potentially sub-optimal, fallback.
*   **Data Rollback/Sanitization:** If data integrity is compromised, attempt to quarantine or rollback to a clean data state, or apply specific sanitization techniques.
*   **Version Control for Policies:** All policies and framework configurations are under version control, allowing easy reversion to previous stable versions.
*   **Continuous Monitoring Post-Rollback:** Intensified monitoring of the rollback policy and data until confidence is restored and a new stable policy is selected.

---

### **5. Ethical, Legal, and Social Implications (ELSI)**

**Stakeholders:**
*   **Decision-makers:** Responsible for policy selection and outcomes.
*   **Impacted Individuals/Groups:** Those directly affected by the policy's implementation (e.g., customers, citizens, employees).
*   **Adversaries:** Entities attempting to manipulate the system (indirectly affected by countermeasures).
*   **Regulators/Oversight Bodies:** Enforcing legal and ethical guidelines.
*   **Developers/Operators:** Building and maintaining the system.

**Potential Biases:**
*   **Scenario Definition Bias:** Scenarios may inadvertently overlook critical edge cases or vulnerabilities, especially for marginalized groups, or be biased by the limited imagination/experience of experts. Adversarial scenarios might over-emphasize known threats and miss novel ones.
*   **Performance Metric Bias:** The chosen metric $M$ might implicitly favor certain outcomes or groups over others, failing to capture holistic impacts or being susceptible to "goodhart's law" (when a measure becomes a target, it ceases to be a good measure).
*   **Data Collection Bias:** Historical data used for scenario calibration or monitoring might reflect existing societal biases, leading to policies that perpetuate inequities. Partially observable data exacerbates this as critical variables might be missing or skewed.
*   **Expert Elicitation Bias:** Experts' judgments can be influenced by personal biases, overconfidence, or limited perspectives.
*   **Rollback Bias:** Default/fallback policies might not be equally fair or effective for all groups, potentially exacerbating existing biases during crisis.

**Concrete Mitigation Strategies:**

1.  **Diverse Scenario Teams & Red Teaming:**
    *   **Strategy:** Form scenario construction teams with diverse backgrounds, expertise, and perspectives. Actively involve "red teams" (including those with adversarial mindsets or from marginalized groups) to identify overlooked vulnerabilities, biases, and adversarial attack vectors. Conduct "pre-mortems" to identify ways the system could fail.
    *   **Mitigation:** Reduces scenario definition bias and enhances the robustness against novel adversarial threats and ethical blind spots.

2.  **Multi-Objective & Equity-Aware Performance Metrics:**
    *   **Strategy:** Define performance using a set of weighted metrics, explicitly including fairness metrics (e.g., disparate impact, equal opportunity) alongside operational metrics. Ensure transparent justification for weighting. Conduct regular audits of these metrics across different demographic or critical sub-groups.
    *   **Mitigation:** Addresses performance metric bias and ensures equitable outcomes are considered during policy selection.

3.  **Data Audits and Bias Detection Tools:**
    *   **Strategy:** Implement automated and manual processes to audit data sources for representational biases, historical inequities, and adversarial tampering. Use fairness-aware AI tools to detect and quantify biases in data distributions. Augment limited data with synthetic, debiased data or transfer learning from more diverse datasets, when appropriate and safe.
    *   **Mitigation:** Identifies and potentially corrects data collection bias, making the information informing scenarios more reliable and equitable.

4.  **Transparent Decision Records & Justification:**
    *   **Strategy:** Maintain a clear, human-readable log of all scenarios considered, their assessed impacts, and the explicit rationale for the chosen policy ($p^*$ minimizes MaxRegret across $S$). This includes documentation of expert inputs and their justifications.
    *   **Mitigation:** Provides interpretability and accountability, allowing stakeholders to understand *why* a particular policy was chosen, facilitating ethical review and legal compliance.

5.  **Stakeholder Engagement & Ethical Review Board:**
    *   **Strategy:** Establish an independent ethical review board (ERB) composed of ethicists, legal experts, community representatives, and technical experts. This board reviews scenarios, performance metrics, and policy selections, especially during critical escalations. Engage affected communities in understanding policy impacts.
    *   **Mitigation:** Ensures broad ethical, legal, and social considerations are integrated into the framework, providing external oversight and legitimacy.

---

### **6. Justification, Limitations, and Failure Scenarios**

**Justification for Practical Performance:**

The RSBPS framework is likely to perform well in the specified context due to its inherent design principles:
1.  **Robustness to Uncertainty:** By focusing on minimax regret across a wide range of scenarios (including extreme and adversarial ones), it explicitly guards against worst-case outcomes rather than hoping for best-case predictions. This is crucial when uncertainty is extreme, and predictions are unreliable.
2.  **Interpretability:** The rationale for selecting a policy is derived directly from its performance across well-defined scenarios. This transparency allows stakeholders to understand the trade-offs and risks, fostering trust and facilitating human oversight and intervention.
3.  **Resilience to Adversarial Data:** Incorporating adversarial scenarios directly into the decision-making process forces the selection of policies that are inherently more resilient to manipulation, rather than attempting to filter out all adversarial data (which is often an impossible task).
4.  **Limited Data & Model Agnosticism:** It does not require vast amounts of historical data for complex model training. Instead, it leverages available data to inform scenario definition, and expert knowledge to assess policy outcomes within those scenarios. This avoids overfitting to sparse or noisy data.
5.  **Built-in Monitoring & Adaptability:** The comprehensive monitoring protocol, combined with explicit re-evaluation triggers and rollback procedures, ensures that the system can detect changes in the environment (including adversarial attacks or concept drift) and adapt the policy accordingly, preventing prolonged periods of sub-optimal or harmful operation.

**Key Limitations:**

1.  **Scenario Completeness:** The framework's effectiveness heavily relies on the completeness and relevance of the defined scenarios. Overlooking critical scenarios or underestimating adversarial capabilities can lead to selecting a sub-optimal or vulnerable policy.
2.  **Expert Elicitation Burden & Bias:** Defining scenarios and evaluating policy outcomes within them requires significant input from domain experts. This can be time-consuming, costly, and susceptible to expert biases or limited imagination.
3.  **Computational Complexity (for large P or S):** If the number of candidate policies ($k$) or scenarios ($m$) is very large, the evaluation step can become computationally intensive, especially if complex simulations are required for each $M(p_i, s_j)$.
4.  **Metric Definition Challenge:** Defining a universally agreed-upon and unbiased performance metric $M$ can be challenging, especially when dealing with complex socio-technical systems.
5.  **No Guarantee of Optimal Performance:** Minimax regret is a pessimistic strategy; it aims to minimize the *maximum possible loss*, not to achieve the *best possible average outcome*. In environments that turn out to be consistently stable and benign, a more optimistic approach might yield better average performance.

**Scenarios Where It May Fail:**

1.  **Unforeseeable Black Swans:** If a truly novel and impactful "black swan" event occurs that was entirely outside the scope of *any* conceivable scenario (even adversarial ones), the selected policy may be ill-equipped, and the monitoring system might not immediately identify it.
2.  **Incapacitated Experts/Lack of Domain Knowledge:** If there are no reliable domain experts available to help define scenarios and assess policy outcomes, or if their knowledge base is fundamentally flawed or outdated, the framework cannot be effectively initialized or maintained.
3.  **Complete Observability Breakdown:** If the environment becomes completely unobservable (e.g., all data streams fail, are utterly corrupted, or are entirely under adversarial control), monitoring becomes impossible, and the framework loses its ability to react or adapt.
4.  **Sophisticated, Undetectable Adversary:** An adversary with unlimited resources and novel, zero-day attack capabilities that completely bypass all known detection mechanisms and were not envisioned in any adversarial scenario could consistently degrade performance without triggering a response.
5.  **Decision Paralysis from Too Many Scenarios:** If the number of scenarios or policies becomes unmanageably large, or if consensus cannot be reached on scenario relevance or performance metrics, the framework could become too slow or complex to implement effectively.

---

In [94]:
# Competitor 3

groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "openai/gpt-oss-120b"

c3_start_time = time.time()

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

c3_end_time = time.time()
c3_duration = c3_end_time - c3_start_time

print(f"Time taken for {model_name}: **{c3_duration:.4f} seconds**\n")
display(Markdown(answer))

competitors.append(model_name)
answers.append(answer)
durations.append(c3_duration)

Time taken for openai/gpt-oss-120b: **6.0449 seconds**



## 1. Model‑agnostic Decision Rule  
**Robust Policy‑Selection (RPS)** – choose the policy whose *worst‑case* expected utility, estimated from the data, is highest.  

| Step | Action | Rationale |
|------|--------|-----------|
| 1️⃣| **Define a finite policy set** 𝒫 = {π₁,…,π_K}. | Keeps the problem tractable and interpretable. |
| 2️⃣| **Collect a limited‑size batch** D = { (o_t, a_t, r_t) } where o_t are observable cues, a_t the action taken, r_t the noisy reward. | Works with sparse data; no need for full model. |
| 3️⃣| **Bootstrap‑aggregate utility estimates**: for each π_k, draw B ≤ 200 bootstrap resamples of D, compute the average reward ˆU_k^b on each resample, and retain the **α‑lower‑confidence bound** L_k = 𝑄_{α}( {ˆU_k^b} ), e.g. α = 0.05. | Guarantees a pessimistic estimate that protects against over‑fitting and noise. |
| 4️⃣| **Apply a simple regularizer**: L_k ← L_k – λ·|π_k| (policy complexity, e.g. number of decision rules). | Favors parsimonious policies → interpretability. |
| 5️⃣| **Select** 𝜋* = arg max_{π_k∈𝒫} L_k. | Maximizes the worst‑case plausible performance. |
| 6️⃣| **Provide rationale**: report L_k, bootstrap distribution, and complexity penalty for the chosen π*. | Transparent justification for stakeholders. |

*Implementation notes* – The algorithm needs only reward samples, no model fitting. Bootstrapping is O(K·B·|D|) and can be run on a single CPU.  

---

## 2. Assumptions, Data Requirements & Empirical Test Plan  

| Assumption | What it means | Data needed | Test (hypothetical) |
|------------|---------------|------------|----------------------|
| **A1 – Partial observability** | We only see a noisy proxy o_t of the true state. | Observations o_t, actions a_t, rewards r_t. | Simulate a hidden Markov process; compare estimated L_k with ground‑truth utility. |
| **A2 – Bounded noise** | Reward noise is sub‑Gaussian with known variance proxy σ². | Repeated reward measurements for same (o,a). | Inject Gaussian noise of variance σ² into synthetic rewards; verify coverage of bootstrap L_k (≈95%). |
| **A3 – Limited adversarial contamination** | ≤ ε % of data may be arbitrarily corrupted. | Flagged or suspect records (optional). | Replace ε % of rewards with extreme outliers; check that selected policy remains unchanged if λ is sufficiently large. |
| **A4 – Finite policy set captures near‑optimal actions** | True optimal policy is approximated by some π_k ∈ 𝒫. | Policy library constructed a‑priori (e.g., decision trees of depth ≤ 3). | In simulated environments, generate optimal policy and verify it lies within distance ε of a library member. |

*Empirical plan*: For each regime (A–C) generate 10 synthetic datasets (|D|=200) and run the RPS pipeline. Record (i) selection stability, (ii) coverage of true utility by L_k, and (iii) false‑positive rate of adversarial detection.

---

## 3. Falsifiable Predictions  

| Regime | Prediction (testable) |
|--------|-----------------------|
| **A – Stable DG with modest noise** | *P1*: The selected policy’s true expected reward will be within 5 % of the oracle optimal reward in ≥ 80 % of simulations. |
| **B – Non‑stationarity / concept drift** | *P2*: After a drift point, the bootstrap‑based L_k will drop by > δ (e.g., 0.2) for the previously optimal policy, triggering a policy change in ≤ 5 % of runs (i.e., low false‑alarm). |
| **C – Adversarial manipulation** | *P3*: When ≥ ε = 10 % of rewards are replaced by adversarial values, the chosen policy will remain the same as in the clean data case in ≥ 90 % of runs (robustness). |

If any prediction is violated, the corresponding assumption (A1‑A4) is falsified and the framework must be revised.

---

## 4. Monitoring & Update Protocol  

| Component | Metric / Test | Trigger Condition | Action |
|-----------|---------------|-------------------|--------|
| **Distributional shift** | Kolmogorov‑Smirnov test on o_t windows (size 50) or ADWIN on reward stream. | p‑value < 0.01 **or** drift magnitude > Δ (e.g., 0.15). | Freeze current policy, recompute bootstrap L_k on the most recent window, select new π*. |
| **Data integrity** | % of rewards beyond 3σ of rolling median; checksum anomalies. | > ε_int = 5 % anomalies. | Flag data batch, invoke manual review; if confirmed, discard batch and re‑run RPS on clean data. |
| **Bias detection** | Disparity in L_k across protected groups (e.g., Δ_L > 0.1). | Detected bias persists over 2 consecutive windows. | Apply group‑wise regularization λ_g, re‑evaluate; if still biased, invoke escalation. |
| **Escalation** | Tier‑1: Automated retraining. Tier‑2: Human‑in‑the‑loop review (policy ethics board). Tier‑3: Full rollback to previously validated policy π_{prev}. |
| **Rollback** | If new policy’s L_k < L_{prev} – γ (γ = 0.05) after 30 new samples, revert to π_{prev} and log incident. |

All alerts and actions are logged with timestamps for auditability.

---

## 5. Ethical, Legal & Social Assessment  

| Issue | Stakeholder impact | Mitigation |
|-------|-------------------|------------|
| **Fairness** | Unequal utility across demographic groups. | Include group‑wise penalties in λ·|π|, monitor Δ_L, and enforce a fairness threshold (e.g., 80 % parity). |
| **Transparency** | Decision makers need to understand why a policy was chosen. | Provide the full bootstrap distribution, complexity penalty, and a concise natural‑language explanation for π*. |
| **Privacy** | Observations may contain personal data. | Store only aggregated statistics, apply differential‑privacy noise (ε≈1) to reward counts before bootstrapping. |
| **Liability** | Wrong policy could cause harm (e.g., medical triage). | Keep a versioned audit trail; maintain a “safe fallback” policy vetted by regulators. |
| **Adversarial exploitation** | Attackers could poison data to force a sub‑optimal policy. | Limit ε‑contamination assumption, monitor integrity metrics, and enforce rate‑limiting on data ingestion. |
| **Regulatory compliance** | GDPR, FDA, etc. | Conduct Data Protection Impact Assessment (DPIA), ensure explainability per EU AI Act, and submit the decision rule to the relevant oversight body. |

---

## 6. Justification, Strengths & Limitations  

**Why RPS should work in practice**  
* **Robustness** – By basing the decision on a lower‑confidence bound, we protect against sampling noise, limited data, and modest adversarial contamination.  
* **Model‑agnosticism** – No need to specify a dynamics model; the method works with any observable‑reward data.  
* **Interpretability** – The final rule is a simple policy (e.g., a shallow decision tree) with an explicit utility bound, easy to audit.  
* **Computationally cheap** – Bootstrap + simple penalty scales linearly with the number of policies; feasible on edge devices.  

**Key limitations**  
1. **Policy library dependence** – If the optimal action lies outside 𝒫, performance degrades (violates A4).  
2. **Bootstrap reliability under extreme heavy‑tailed noise** – Sub‑Gaussian assumption may fail, leading to overly optimistic L_k.  
3. **Detection latency** – Drift tests need a few dozen samples; rapid shifts may cause temporary sub‑optimal actions.  
4. **Adversarial budget** – The guarantee holds only up to the assumed ε contamination; a coordinated attack exceeding this budget can break the system.  

**Failure scenarios**  
* Highly non‑stationary environments where the reward distribution changes faster than the monitoring window.  
* When observation space o_t is so severely corrupted that no informative signal remains (complete loss of partial observability).  

---

## Final Decision Rule (Executable Pseudocode)

```python
# Robust Policy Selection (RPS)

import numpy as np
from sklearn.utils import resample

def rps(decision_policies, data, B=150, alpha=0.05, lam=0.01):
    """
    decision_policies : list of callables π_k(o) -> a
    data             : list of tuples (o, a, r)
    B                : number of bootstrap replications
    alpha            : lower‑confidence level (e.g., 0.05)
    lam              : complexity penalty weight
    Returns (π_star, L_star, diagnostics)
    """
    # pre‑compute policy complexities (e.g., number of rules)
    complexities = [policy.complexity() for policy in decision_policies]

    # extract rewards per policy
    rewards_by_policy = {k: [] for k in range(len(decision_policies))}
    for (o, a, r) in data:
        for k, π in enumerate(decision_policies):
            if π(o) == a:                # reward belongs to policy k
                rewards_by_policy[k].append(r)

    # bootstrap lower bounds
    lower_bounds = []
    for k, rewards in rewards_by_policy.items():
        if len(rewards) == 0:
            lower_bounds.append(-np.inf)
            continue
        boot_means = []
        for _ in range(B):
            sample = resample(rewards, replace=True, n_samples=len(rewards))
            boot_means.append(np.mean(sample))
        Lk = np.quantile(boot_means, alpha) - lam * complexities[k]
        lower_bounds.append(Lk)

    # choose best policy
    best_idx = int(np.argmax(lower_bounds))
    π_star = decision_policies[best_idx]
    L_star = lower_bounds[best_idx]

    diagnostics = {
        "lower_bounds": lower_bounds,
        "complexities": complexities,
        "bootstrap_means": boot_means  # last computed (optional)
    }
    return π_star, L_star, diagnostics
```

*Deploy*: run `rps` after every monitoring window (≈50 new samples). If the selected `π_star` differs from the current operational policy, trigger the **Update Protocol** (Section 4).  

---  

*End of answer.*

In [96]:
print(competitors)
print(durations)

['gpt-5-mini', 'gemini-2.5-flash', 'openai/gpt-oss-120b']
[63.00679039955139, 39.52344059944153, 6.044901609420776]


In [100]:
# Building DataFrame

import pandas as pd

competitors_answers = pd.DataFrame({
    "Model": competitors,
    "Answer": answers,
    "Time (s)": [round(d, 3) for d in durations]
})

competitors_answers

,Model,Answer,Time (s)
0,gpt-5-mini,# Robust Decision Framework for Selecting a Si...,63.007
1,gemini-2.5-flash,The following robust decision framework is des...,39.523
2,openai/gpt-oss-120b,## 1. Model‑agnostic Decision Rule \n**Robust...,6.045


In [104]:
# FINAL JUDGING STATE
# Combine all answers nicely
together = "\n\n".join(
	f"### Competitor {i + 1}: **{competitors[i]}**\n{answer}"
	for i, answer in enumerate(answers)
)

display(Markdown(together))

### Competitor 1: **gpt-5-mini**
# Robust Decision Framework for Selecting a Single Policy under Extreme Uncertainty

Summary: a model-agnostic, implementable rule (Robust Lower-bound Selection, RLS) that (a) evaluates candidate policies under an ensemble of plausible perturbations (noise, shifts, adversarial manipulations), (b) chooses the policy that maximizes a conservative performance metric (worst-case lower confidence bound or CVaR) while penalizing complexity, (c) produces an interpretable rationale and monitoring/rollback plan.

---

## 1. Final decision rule (algorithm: Robust Lower-bound Selection — RLS)

Inputs
- Candidate policy set P = {π1,...,πk} (can be rules, parametric policies, or black-box policies).
- History dataset D = {(x_i, a_i, r_i, t_i, meta_i)} (covariates x, action a, reward/outcome r, timestamp t, metadata).
- Baseline policy π0 (safe fallback).
- Utility function U(π; D) (e.g., expected reward, harm-weighted utility).
- Uncertainty generators G = {g1,...,gm} describing plausible perturbations (noise levels, covariate shifts, label corruption, adversarial fraction ε).
- Off-policy evaluator E (IPS with clipped weights or Doubly Robust), bootstrap count B, confidence level α (e.g., 0.05), CVaR tail τ (optional, e.g., 0.1).
- Complexity penalty C(π) (e.g., model size, interpretability cost), weight λ.
- Trigger thresholds for monitoring (see §4).

Outputs
- Selected policy π* and interpretability report R (LCBs/CVaR across scenarios, sensitivity summary, failure modes).
- Deployment plan & rollback steps.

Procedure
1. Data sanitization:
   - Basic integrity checks (missingness, duplicates, signature/hashes), remove/flag corrupted rows.
   - Estimate propensity scores p(a|x) if needed for IPS; apply clipping at w_max.

2. Build perturbation scenarios:
   - For each g ∈ G, define parameterized perturbations (e.g., additive noise σ, covariate shift magnitude δ, label flip rate η, fraction ε of adversarial examples).
   - Use domain knowledge to bound ranges (conservative priors).

3. Bootstrap evaluation:
   - For b = 1..B:
     - Resample D_b from D (with stratification if needed).
     - For each scenario s generated from G:
       - Apply perturbation s to D_b → D_{b,s}.
       - For each candidate policy π:
         - Compute off-policy estimate U_{b,s}(π) using E on D_{b,s}; record estimator variance.
   - This yields distribution of U across (b,s) for each π.

4. Compute robust score per π:
   - Option A (Lower-confidence bound): LCB(π) = empirical α-quantile of U_{b,s}(π) across all (b,s).
   - Option B (CVaR): CVaR_τ(π) = mean of worst τ fraction of U_{b,s}(π).
   - RobustScore(π) = LCB(π) − λ·C(π) (or CVaR_τ(π) − λ·C(π)).
   - Optionally compute worst-case regret relative to baseline: WorstRegret(π) = max_{b,s} (U_{b,s}(π0) − U_{b,s}(π)).

5. Selection:
   - Choose π* = argmax_π RobustScore(π), subject to constraint WorstRegret(π) ≤ R_max (set R_max per risk tolerance). If no π meets constraints, default to baseline π0.

6. Rationale and interpretability output R:
   - Report LCB/CVaR and median for chosen π*, complexity cost, most damaging perturbation scenarios, sensitivity charts, and policy decision rule summary.
   - Provide simple explanation: e.g., "π* selected because its 95% lower bound on expected utility under plausible perturbations is greatest and worst-case regret vs baseline ≤ X."

7. Deploy with guardrails:
   - Deploy π* in staged rollout (canary, A/B with heavy monitoring), see §4 for triggers and rollback.

Implementation notes
- Off-policy evaluation: use Doubly Robust or weighted IPS; clip importance weights and report sensitivity to clipping.
- Keep candidate policies simple where possible (rules, small trees, linear policies) to improve interpretability and reduce overfitting.
- Choose B (e.g., 500) and number of scenarios m pragmatically—enough for stable estimates given data size.

---

## 2. Assumptions, data requirements, and empirical test plan

Assumptions (explicit)
- A1: Utility function U correctly captures stakeholder objectives (or a defensible proxy exists).
- A2: Candidate policies P contain at least one safe fallback π0.
- A3: Historical data D is informative for the short-term behavior of the system (coverage of relevant covariate-action space).
- A4: Perturbation family G contains realistic bounds for noise, shift, and adversary strength (conservative, but plausible).
- A5: Off-policy evaluation bias is tolerable with clipping/DR corrections (no extreme propensities).

Data requirements (minimum)
- N_total: at least a few hundred independent episodes per key stratum; if scarce, rely more on conservative G and simpler policies.
- Metadata: timestamps, provenance, and integrity checks.
- Feature coverage: key covariates that influence action-outcome relationships must appear in D.
- Baseline behavior data for π0 to compute regret.

Plan to empirically test assumptions (hypothetical scenarios)
- Create synthetic datasets that mimic domain features and labeled outcomes; vary noise, shift, adversarial fractions.
- Tests:
  - Coverage test (A3): remove certain covariate strata from training data; measure IPS variance and LCB widening.
  - Sensitivity to G (A4): expand/shrink perturbation ranges; confirm selection stability (π* stable under reasonable changes).
  - OPE validity (A5): compare off-policy estimates to on-policy returns in small controlled online rollouts (canary) to validate E calibration.
- Acceptance criteria:
  - If empirical LCBs consistently underestimate true on-policy canary returns by >δ, recalibrate E or tighten rolling deployment.
  - If candidate selection flips across many plausible G changes, increase conservatism (larger λ or stricter WorstRegret).

---

## 3. Falsifiable predictions (three regimes)

Define metric M = realized average utility per episode measured on holdout/on-policy canary.

Regime A — Stable, modest noise
- Prediction A1: Within T_A (e.g., 2 weeks or 10k episodes), M(π*) ≥ median LCB(π*) + ε_A (ε_A small, e.g., 0.02 utility units).
- Falsifiability: if M(π*) < LCB(π*) − δ_A, prediction fails.

Regime B — Non-stationarity/concept drift
- Prediction B1: M(π*) will degrade gradually; degradation slope s satisfies |ΔM/Δt| ≤ s_max over initial window W (we expect detect-and-update triggers before performance drops below baseline).
- Falsifiability: if M(π*) drops abruptly below baseline π0 without preceding drift signals (distributional detectors not triggered), prediction fails.

Regime C — Adversarial manipulation
- Prediction C1: Under adversarial fraction ≤ ε_adversary (modeled in G), worst-case observed M(π*) ≥ WorstCase_LCB(π*) − ε_C (conservative), and monitoring will detect integrity anomalies (metadata tampering, abnormal reward patterns) within detection window T_C.
- Falsifiability: if adversary-induced outcomes produce M(π*) significantly below WorstCase_LCB and no integrity flags are raised in T_C, prediction fails.

Each prediction is falsifiable by comparing on-policy logged metrics to predicted intervals and by inspecting detector triggers.

---

## 4. Monitoring and update protocol (detection, escalation, rollback)

Continuous monitoring components
- Data-distribution monitors:
  - Covariate shift: population stability index (PSI) and KL divergence on key features; threshold θ_cov (e.g., PSI > 0.2).
  - Label/Outcome shift: compare recent outcome distribution to baseline with Earth Mover’s Distance; threshold θ_lab.
- Model/performance monitors:
  - Utility drift: rolling average of utility M with statistical control charts (CUSUM/Shewhart) at significance α_mon.
  - Off-policy vs on-policy divergence: compare OPE predictions to canary actuals.
- Integrity/anomaly monitors:
  - Metadata validation (signatures, device IDs), outlier detection on timestamps and event rates.
  - Adversarial detectors: sudden cluster of contradictory examples, reward spikes, or disproportionate error rates.
- Fairness/bias monitors:
  - Disparate impact (ratio metrics), subgroup utilities, intersectional checks; thresholds set by policy.

Trigger conditions (examples)
- Tier 1 (warning): PSI > θ1 or utility drift p-value < 0.05 → notify operations and data science.
- Tier 2 (action required): PSI > θ2 or utility drop > X% vs baseline OR integrity anomaly → pause new exposures to π*; initiate canary rollback to π0 for affected segments.
- Tier 3 (urgent rollback): evidence of data tampering or targeted harm (meet legal/ethical stop criteria) → immediate full rollback to π0 and incident response.

Escalation path
- Tier 1: Automated alert → data science on-call inspects dashboards within 4 hours.
- Tier 2: Convene cross-functional response (ops, DS, legal, compliance) within 2 hours; restrict traffic to canary; collect forensic logs.
- Tier 3: Immediate cessation; notify leadership and regulators as required; start post-incident review.

Rollback & recovery procedures
- Pre-deploy: ensure π0 is always deployable, versioned, and tested; keep feature transforms consistent.
- Immediate rollback: switch routing to π0; preserve logs; mark affected data for quarantine.
- Post-rollback: root-cause analysis, patch G and monitoring thresholds, retrain or replace candidate set if needed.
- Audit trail: full logging of decision, metrics, time of rollback, and rationale; share with stakeholders.

---

## 5. Ethical, legal, and social implications & mitigation

Key stakeholder impacts
- Users potentially harmed by mistaken decisions (safety-critical domains).
- Subpopulations disproportionately affected (bias).
- Data subjects’ privacy and consent.
- Organizational/legal exposure (regulation compliance).

Risks
- Distributional bias causing disproportionate harms.
- Adversarial exploitation leading to targeted harms.
- Over-reliance on automated decision with insufficient human oversight.
- Privacy leaks from logs or models.

Concrete mitigation strategies
- Fairness constraints: enforce minimum subgroup utility floors during selection (add constraints to WorstRegret or per-group LCB≥g_min).
- Transparency & explanation: produce R reporting interpretable rationale for chosen policy; publish summary for stakeholders.
- Human-in-the-loop: require human approval for high-risk deployments and for Tier 2+ escalations.
- Data governance: strict provenance, access controls, hashing/signatures for integrity, and retention policies.
- Privacy protections: minimize sensitive attributes in decisioning; apply differential privacy or noise to analytics when needed.
- Adversarial hardening: red-team tests, simulated adversary scenarios in G, input sanity checks.
- Legal compliance: consult counsel for sector rules (e.g., GDPR, HIPAA); implement opt-out and data subject rights flows.
- Independent audits: periodic external fairness/safety audits and public reporting where appropriate.

---

## 6. Justification, expected performance, and limitations

Why this will perform well in practice
- Conservative selection (LCB/CVaR) reduces chance of worst-case catastrophic outcomes under uncertainty—suitable when stakes are high.
- Model-agnostic: works with rules, simple models, or complex policies; off-policy evaluation avoids extensive online exploration.
- Bootstrapping + scenario perturbations produce robust uncertainty quantification even with limited data.
- Complexity penalty and preference for simpler policies reduce overfitting and improve interpretability.

Key limitations and failure modes
- Conservatism: may select suboptimal policy for benign environments (opportunity cost).
- Unmodeled "unknown unknowns": if real-world perturbations lie outside G, selection may fail.
- Dependence on OPE accuracy: heavy bias in E (poor propensities, extreme covariate mismatch) undermines estimates.
- Candidate-policy coverage: if P lacks any robust good policy, selection cannot fix it—requires human design loop.
- Data scarcity: with very small N, LCBs are wide and decision may default to baseline frequently.
- Sophisticated adversary: if adversary can adapt in deployment beyond modeled capacity, monitoring might lag.

Mitigations for limitations
- Periodic expansion of G via adversarial red-teaming; incorporate rapid canary cycles to validate OPE.
- Maintain policy development pipeline to generate new candidates when selection repeatedly returns π0.
- Combine with conservative online learning only under strict safety constraints.

---

## Quick parameter guidance (practical defaults)
- B (bootstrap): 200–1000 depending on compute.
- α for LCB: 0.05 (95% lower bound).
- CVaR τ: 0.1 if risk-averse.
- Importance weight clipping w_max: 10.
- Complexity penalty λ: tuned so simple policy preferred unless complex policy improves LCB by >δ.
- Monitoring windows: short-term (hourly) for integrity; medium-term (daily) for utility drift.

---

## Deliverables produced by the framework
- Selected policy π* with decision report R (LCBs, worst-case scenarios, sensitivity).
- Monitoring dashboard with PSI, utility charts, fairness metrics, integrity logs.
- Escalation playbook and rollback scripts (automated routing to π0).
- Test suite of simulated scenarios used to validate assumptions.

---

This framework is concise, implementable with limited data, and designed to minimize overfitting while producing interpretable justification for the chosen policy. It trades some short-term performance for robustness and safety under extreme uncertainty; use domain expertise to set G, thresholds, and the baseline policy.

### Competitor 2: **gemini-2.5-flash**
The following robust decision framework is designed for selecting a single policy under extreme uncertainty, partial observability, and with noisy/adversarial data.

---

### **1. Model-Agnostic Decision Rule: Robust Scenario-Based Policy Selection (RSBPS)**

The RSBPS framework prioritizes robustness and interpretability over predictive accuracy, which is highly challenging under extreme uncertainty and adversarial conditions. It identifies a policy that minimizes the maximum regret across a diverse set of plausible, stressful, and adversarial future scenarios.

**Input:**
*   A finite set of candidate policies: $P = \{p_1, p_2, \dots, p_k\}$.
*   A set of critical uncertainties and features derived from the partially observable, noisy data: $U = \{u_1, u_2, \dots, u_n\}$. These represent key variables or processes that significantly impact policy outcomes but are subject to high uncertainty or manipulation.
*   A performance metric: $M(p_i, s_j)$, a scalar value representing the outcome (e.g., cost, safety, efficiency, impact) of applying policy $p_i$ under scenario $s_j$. Assume $M$ is defined such that *lower values are better* (e.g., lower cost, fewer failures). If higher values are better, adjust regret calculation accordingly.

**Algorithm:**

1.  **Scenario Construction (S):**
    *   **Define Plausible & Adversarial Scenarios:** Based on current and historical data, expert knowledge, and known adversarial tactics, construct a comprehensive set of future scenarios $S = \{s_1, s_2, \dots, s_m\}$. Each $s_j$ describes a specific realization of the critical uncertainties $U$.
    *   **Types of Scenarios:**
        *   **Nominal:** Most likely or expected conditions (e.g., stable data-generating process with modest noise).
        *   **Stress-Test:** Extreme but plausible conditions (e.g., significant but random data noise, unexpected market shifts, resource scarcity).
        *   **Adversarial:** Conditions where data is actively manipulated or system behavior is targeted (e.g., data poisoning, spoofing, denial-of-service, concept drift induced by an adversary).
        *   **Drift/Non-stationarity:** Scenarios explicitly modeling shifts in data distributions or underlying processes.
        *   **Black Swan (if conceivable):** Highly improbable but high-impact events.
    *   *Rationale:* This step integrates domain expertise and adversarial thinking directly into the decision process, bypassing the need for complex predictive models.

2.  **Policy Performance Evaluation:**
    *   For each policy $p_i \in P$ and each scenario $s_j \in S$, estimate or simulate the performance metric $M(p_i, s_j)$. This step may involve simplified models, expert elicitation, or historical data-driven simulations tailored to each scenario's conditions.
    *   *Rationale:* Directly assesses policy outcomes under diverse conditions, rather than relying on a single, potentially flawed model of the environment.

3.  **Regret Calculation:**
    *   For each scenario $s_j$, identify the *optimal* performance among all candidate policies: $M_{optimal}(s_j) = \min_{p_i \in P} \{M(p_i, s_j)\}$.
    *   Calculate the *scenario-specific regret* for each policy $p_i$ under scenario $s_j$: $R(p_i, s_j) = M(p_i, s_j) - M_{optimal}(s_j)$. This measures how much worse policy $p_i$ performs compared to the best possible policy *for that specific scenario*.
    *   *Rationale:* Focuses on relative performance, which is more robust than absolute performance in highly uncertain environments.

4.  **Max Regret Identification:**
    *   For each policy $p_i$, identify its *maximum regret* across all scenarios: $MaxRegret(p_i) = \max_{s_j \in S} \{R(p_i, s_j)\}$. This represents the worst-case outcome for policy $p_i$ relative to the optimal choice for any given scenario.
    *   *Rationale:* Explicitly identifies and quantifies the "least worst" performance, preparing for pessimistic outcomes.

5.  **Policy Selection (Minimax Regret Criterion):**
    *   Select the policy $p^*$ that minimizes the maximum regret:
        $p^* = \arg \min_{p_i \in P} \{MaxRegret(p_i)\}$
    *   *Rationale:* This "minimax regret" strategy selects the policy that guarantees the best possible outcome under the worst possible scenario (relative to the optimal choice for that scenario). It's a robust choice when facing deep uncertainty and potential adversaries.

**Interpretable Rationale:** The selected policy $p^*$ is chosen because it is the most robust option, minimizing the maximum potential "missed opportunity" or "sub-optimality" across all considered scenarios. Its rationale is directly tied to its comparative performance across the explicitly defined plausible, stressful, and adversarial futures.

---

### **2. Assumptions, Data Requirements, and Empirical Testing Plan**

**Assumptions:**
*   **Existence of Candidate Policies:** A finite, well-defined set of policies exists for consideration.
*   **Feasibility of Scenario Definition:** It's possible to define a sufficiently rich and relevant set of scenarios (including adversarial ones) that capture the critical uncertainties.
*   **Measurable Performance:** A quantifiable, agreed-upon performance metric $M$ can be defined for any policy under any scenario.
*   **Bounded Rationality of Adversaries (for adversarial scenarios):** While adversaries are malicious, their capabilities and objectives are not entirely unknowable; there are limits to their manipulation.
*   **Observable Indicators (even if noisy):** Despite partial observability and noise, some indicators or features (e.g., changes in correlation, distribution shifts, specific attack signatures) can be detected to inform scenario updates and monitoring.

**Data Requirements:**
*   **Historical Policy Performance Data:** Records of how past policies performed under varying conditions (if available).
*   **Feature Data:** Current and historical data streams relevant to the critical uncertainties $U$, even if noisy or sparse. This informs scenario definition and monitoring.
*   **Expert Elicitation:** Input from domain experts on plausible scenarios, potential adversarial tactics, and policy outcomes in specific situations.
*   **Adversarial Intelligence:** Data on known attack vectors, adversarial capabilities, and objectives from security reports or threat intelligence.
*   **Policy Specifications:** Detailed descriptions of each candidate policy $p_i$.

**Empirical Testing Plan (Hypothetical Scenarios):**

1.  **Regime A: Stable Data-Generating Process with Modest Noise**
    *   **Hypothetical Setup:** Simulate a stable environment where underlying processes are constant, and data noise follows a well-behaved distribution (e.g., Gaussian). Candidate policies are designed for this environment.
    *   **Test:**
        *   Define scenarios including the nominal stable state and slight variations due to noise.
        *   Run RSBPS to select a policy.
        *   Generate multiple independent data runs from the stable process.
        *   Evaluate the selected policy's performance.
    *   **Expected Outcome:** RSBPS should consistently select a policy close to the empirically optimal policy for the stable regime, demonstrating its ability to find a robust choice even when uncertainty is minimal.

2.  **Regime B: Non-stationarity and Concept Drift**
    *   **Hypothetical Setup:** Introduce a gradual or abrupt shift in the data-generating process (e.g., changing correlations between features, altered underlying distributions) after an initial stable period.
    *   **Test:**
        *   Initially, use RSBPS under the stable regime.
        *   Introduce the concept drift.
        *   Use the monitoring protocol (Section 4) to detect the shift.
        *   Upon detection, re-run RSBPS with *updated scenarios* that incorporate the detected drift.
        *   Compare the performance of the *initial* selected policy versus the *re-selected* policy after drift detection.
    *   **Expected Outcome:** The monitoring system should reliably detect the drift. RSBPS, when re-run with updated scenarios, should select a different, more appropriate policy that performs significantly better in the drifted environment than the initial policy.

3.  **Regime C: Adversarial Manipulation**
    *   **Hypothetical Setup:** Simulate an environment where a subset of input data is maliciously manipulated (e.g., data poisoning, feature injection) by an adversary aiming to degrade system performance or steer policy choice.
    *   **Test:**
        *   Define adversarial scenarios within RSBPS, explicitly modeling the adversary's capabilities and goals.
        *   Run RSBPS.
        *   Simulate the adversarial attacks on the data streams.
        *   Evaluate the selected policy's performance under these attacks, and also evaluate other "less robust" policies (e.g., one selected by a purely predictive model without adversarial scenarios).
    *   **Expected Outcome:** The policy selected by RSBPS should demonstrate greater resilience (i.e., less degradation in performance) against the simulated adversarial attacks compared to policies not chosen with adversarial scenarios in mind. Monitoring should detect integrity issues.

---

### **3. Falsifiable Predictions**

1.  **Regime A (Stable Data-Generating Process with Modest Noise):**
    *   **Prediction:** If the data-generating process is stable with modest noise, the RSBPS framework will select a policy whose average performance over 100 independent trials falls within 5% of the empirically optimal policy's average performance, and the selected policy will remain unchanged for at least 90% of re-evaluations (if no monitoring triggers).
    *   **Falsification:** If the chosen policy's average performance is consistently more than 5% worse than the empirically optimal, or if the policy selection frequently changes without external triggers, the prediction is falsified.

2.  **Regime B (Non-stationarity and Concept Drift):**
    *   **Prediction:** When a significant concept drift (defined as a statistical distribution shift detectable by a Kolmogorov-Smirnov test at p<0.01 between pre- and post-drift data windows) occurs, the monitoring protocol will trigger a re-evaluation within 3 data cycles, leading to RSBPS selecting a *different* policy from the original one in at least 80% of drift events, and this new policy will perform at least 10% better than the original policy would have in the post-drift environment.
    *   **Falsification:** If drift is not detected, or if the selected policy does not change, or if the new policy does not significantly outperform the old one post-drift, the prediction is falsified.

3.  **Regime C (Adversarial Manipulation):**
    *   **Prediction:** If 10% of input data is subjected to known adversarial poisoning attacks (e.g., label flipping, specific feature manipulation), the policy selected by RSBPS (which explicitly considers adversarial scenarios) will experience at most a 15% degradation in its performance metric, whereas a policy selected without considering adversarial scenarios would experience a degradation exceeding 30%. The monitoring system will also flag data integrity issues in over 95% of attack instances.
    *   **Falsification:** If the RSBPS-selected policy degrades by more than 15%, or if the non-adversarially chosen policy degrades by less than 30%, or if the monitoring system fails to detect most attacks, the prediction is falsified.

---

### **4. Monitoring and Update Protocol**

This protocol ensures the continued relevance and safety of the selected policy.

**Key Components:**
*   **Data Quality & Integrity Monitors:**
    *   **Scope:** Raw input data, processed features, intermediary model outputs (if any).
    *   **Checks:** Value range, completeness, consistency, statistical anomalies (e.g., sudden spikes/drops, outliers), feature correlations, entropy levels.
    *   **Techniques:** Control charts (e.g., CUSUM, EWMA), statistical tests (e.g., Z-test for means, Chi-squared for distributions), anomaly detection algorithms (e.g., Isolation Forest, Autoencoders).
*   **Distributional Shift Monitors:**
    *   **Scope:** Key feature distributions, target variable distribution (if applicable), relationship between features and outcomes.
    *   **Checks:** Change in mean, variance, skewness, kurtosis; shifts in multivariate distributions.
    *   **Techniques:** Drift detection algorithms (e.g., ADWIN, DDM), statistical tests (e.g., Kolmogorov-Smirnov, Jensen-Shannon divergence, Adversarial validation), concept drift detectors.
*   **Performance Monitors:**
    *   **Scope:** Actual observed policy performance against the chosen metric $M$.
    *   **Checks:** Deviations from expected performance, increasing variance, unexpected outcomes.
    *   **Techniques:** A/B testing (if multiple policies are run in parallel in sub-segments), backtesting against historical data, control charts on performance metrics.
*   **Bias Monitors:**
    *   **Scope:** Disparate impact across defined demographic groups or critical sub-populations, fairness metrics (e.g., Equal Opportunity, Demographic Parity).
    *   **Checks:** Discrepancies in performance $M$ or other impact metrics across groups.
    *   **Techniques:** Regular auditing of group-specific outcomes.

**Trigger Conditions:**
*   **Severity 1 (Warning):**
    *   Minor statistical anomalies or deviations (e.g., 1-sigma deviation in a control chart).
    *   Small, localized shifts in feature distributions that don't yet impact overall performance.
    *   Slight increase in uncertainty estimates.
    *   *Action:* Automated alert to monitoring team, increased logging, deeper automated diagnostics.
*   **Severity 2 (Alert):**
    *   Persistent or significant statistical anomalies (e.g., 2-sigma deviation).
    *   Detected distributional shift (e.g., K-S test p-value < 0.05).
    *   Observed performance degradation (e.g., $M$ deviates by 5-10% from expected).
    *   *Action:* Immediate alert to human operators/experts, detailed diagnostic report, initiation of a review process.
*   **Severity 3 (Critical):**
    *   Catastrophic data integrity issues (e.g., data source offline, massive corruption).
    *   Major concept drift or non-stationarity (e.g., K-S test p-value < 0.01, ADWIN/DDM triggers).
    *   Significant performance degradation (e.g., $M$ deviates by >10% from expected, or violates safety thresholds).
    *   Detection of known adversarial attack signatures.
    *   *Action:* Emergency protocol initiation, immediate policy review, potential temporary policy rollback.

**Escalation Paths:**
1.  **Automated Anomaly Detection (Severity 1):** System generates internal alerts and logs.
2.  **Human Monitoring Team (Severity 2):** Designated team investigates alerts, performs root cause analysis, and determines if a policy re-evaluation is needed.
3.  **Expert Panel/Stakeholder Review (Severity 3):** If critical issues arise, or a policy change is proposed, a panel of domain experts and key stakeholders (including ethical review board members) convenes to review diagnostics, assess new scenarios, and approve policy re-selection or emergency measures. This includes reviewing potential new adversarial scenarios.
4.  **Policy Re-evaluation:** If drift or adversarial activity is confirmed, the RSBPS framework is re-run with updated scenarios reflecting the new reality, potentially leading to a new policy selection.

**Rollback Procedures:**
*   **Temporary Rollback:** In a critical situation, revert to a previously known "safe" or default policy. This policy should be pre-approved as a stable, albeit potentially sub-optimal, fallback.
*   **Data Rollback/Sanitization:** If data integrity is compromised, attempt to quarantine or rollback to a clean data state, or apply specific sanitization techniques.
*   **Version Control for Policies:** All policies and framework configurations are under version control, allowing easy reversion to previous stable versions.
*   **Continuous Monitoring Post-Rollback:** Intensified monitoring of the rollback policy and data until confidence is restored and a new stable policy is selected.

---

### **5. Ethical, Legal, and Social Implications (ELSI)**

**Stakeholders:**
*   **Decision-makers:** Responsible for policy selection and outcomes.
*   **Impacted Individuals/Groups:** Those directly affected by the policy's implementation (e.g., customers, citizens, employees).
*   **Adversaries:** Entities attempting to manipulate the system (indirectly affected by countermeasures).
*   **Regulators/Oversight Bodies:** Enforcing legal and ethical guidelines.
*   **Developers/Operators:** Building and maintaining the system.

**Potential Biases:**
*   **Scenario Definition Bias:** Scenarios may inadvertently overlook critical edge cases or vulnerabilities, especially for marginalized groups, or be biased by the limited imagination/experience of experts. Adversarial scenarios might over-emphasize known threats and miss novel ones.
*   **Performance Metric Bias:** The chosen metric $M$ might implicitly favor certain outcomes or groups over others, failing to capture holistic impacts or being susceptible to "goodhart's law" (when a measure becomes a target, it ceases to be a good measure).
*   **Data Collection Bias:** Historical data used for scenario calibration or monitoring might reflect existing societal biases, leading to policies that perpetuate inequities. Partially observable data exacerbates this as critical variables might be missing or skewed.
*   **Expert Elicitation Bias:** Experts' judgments can be influenced by personal biases, overconfidence, or limited perspectives.
*   **Rollback Bias:** Default/fallback policies might not be equally fair or effective for all groups, potentially exacerbating existing biases during crisis.

**Concrete Mitigation Strategies:**

1.  **Diverse Scenario Teams & Red Teaming:**
    *   **Strategy:** Form scenario construction teams with diverse backgrounds, expertise, and perspectives. Actively involve "red teams" (including those with adversarial mindsets or from marginalized groups) to identify overlooked vulnerabilities, biases, and adversarial attack vectors. Conduct "pre-mortems" to identify ways the system could fail.
    *   **Mitigation:** Reduces scenario definition bias and enhances the robustness against novel adversarial threats and ethical blind spots.

2.  **Multi-Objective & Equity-Aware Performance Metrics:**
    *   **Strategy:** Define performance using a set of weighted metrics, explicitly including fairness metrics (e.g., disparate impact, equal opportunity) alongside operational metrics. Ensure transparent justification for weighting. Conduct regular audits of these metrics across different demographic or critical sub-groups.
    *   **Mitigation:** Addresses performance metric bias and ensures equitable outcomes are considered during policy selection.

3.  **Data Audits and Bias Detection Tools:**
    *   **Strategy:** Implement automated and manual processes to audit data sources for representational biases, historical inequities, and adversarial tampering. Use fairness-aware AI tools to detect and quantify biases in data distributions. Augment limited data with synthetic, debiased data or transfer learning from more diverse datasets, when appropriate and safe.
    *   **Mitigation:** Identifies and potentially corrects data collection bias, making the information informing scenarios more reliable and equitable.

4.  **Transparent Decision Records & Justification:**
    *   **Strategy:** Maintain a clear, human-readable log of all scenarios considered, their assessed impacts, and the explicit rationale for the chosen policy ($p^*$ minimizes MaxRegret across $S$). This includes documentation of expert inputs and their justifications.
    *   **Mitigation:** Provides interpretability and accountability, allowing stakeholders to understand *why* a particular policy was chosen, facilitating ethical review and legal compliance.

5.  **Stakeholder Engagement & Ethical Review Board:**
    *   **Strategy:** Establish an independent ethical review board (ERB) composed of ethicists, legal experts, community representatives, and technical experts. This board reviews scenarios, performance metrics, and policy selections, especially during critical escalations. Engage affected communities in understanding policy impacts.
    *   **Mitigation:** Ensures broad ethical, legal, and social considerations are integrated into the framework, providing external oversight and legitimacy.

---

### **6. Justification, Limitations, and Failure Scenarios**

**Justification for Practical Performance:**

The RSBPS framework is likely to perform well in the specified context due to its inherent design principles:
1.  **Robustness to Uncertainty:** By focusing on minimax regret across a wide range of scenarios (including extreme and adversarial ones), it explicitly guards against worst-case outcomes rather than hoping for best-case predictions. This is crucial when uncertainty is extreme, and predictions are unreliable.
2.  **Interpretability:** The rationale for selecting a policy is derived directly from its performance across well-defined scenarios. This transparency allows stakeholders to understand the trade-offs and risks, fostering trust and facilitating human oversight and intervention.
3.  **Resilience to Adversarial Data:** Incorporating adversarial scenarios directly into the decision-making process forces the selection of policies that are inherently more resilient to manipulation, rather than attempting to filter out all adversarial data (which is often an impossible task).
4.  **Limited Data & Model Agnosticism:** It does not require vast amounts of historical data for complex model training. Instead, it leverages available data to inform scenario definition, and expert knowledge to assess policy outcomes within those scenarios. This avoids overfitting to sparse or noisy data.
5.  **Built-in Monitoring & Adaptability:** The comprehensive monitoring protocol, combined with explicit re-evaluation triggers and rollback procedures, ensures that the system can detect changes in the environment (including adversarial attacks or concept drift) and adapt the policy accordingly, preventing prolonged periods of sub-optimal or harmful operation.

**Key Limitations:**

1.  **Scenario Completeness:** The framework's effectiveness heavily relies on the completeness and relevance of the defined scenarios. Overlooking critical scenarios or underestimating adversarial capabilities can lead to selecting a sub-optimal or vulnerable policy.
2.  **Expert Elicitation Burden & Bias:** Defining scenarios and evaluating policy outcomes within them requires significant input from domain experts. This can be time-consuming, costly, and susceptible to expert biases or limited imagination.
3.  **Computational Complexity (for large P or S):** If the number of candidate policies ($k$) or scenarios ($m$) is very large, the evaluation step can become computationally intensive, especially if complex simulations are required for each $M(p_i, s_j)$.
4.  **Metric Definition Challenge:** Defining a universally agreed-upon and unbiased performance metric $M$ can be challenging, especially when dealing with complex socio-technical systems.
5.  **No Guarantee of Optimal Performance:** Minimax regret is a pessimistic strategy; it aims to minimize the *maximum possible loss*, not to achieve the *best possible average outcome*. In environments that turn out to be consistently stable and benign, a more optimistic approach might yield better average performance.

**Scenarios Where It May Fail:**

1.  **Unforeseeable Black Swans:** If a truly novel and impactful "black swan" event occurs that was entirely outside the scope of *any* conceivable scenario (even adversarial ones), the selected policy may be ill-equipped, and the monitoring system might not immediately identify it.
2.  **Incapacitated Experts/Lack of Domain Knowledge:** If there are no reliable domain experts available to help define scenarios and assess policy outcomes, or if their knowledge base is fundamentally flawed or outdated, the framework cannot be effectively initialized or maintained.
3.  **Complete Observability Breakdown:** If the environment becomes completely unobservable (e.g., all data streams fail, are utterly corrupted, or are entirely under adversarial control), monitoring becomes impossible, and the framework loses its ability to react or adapt.
4.  **Sophisticated, Undetectable Adversary:** An adversary with unlimited resources and novel, zero-day attack capabilities that completely bypass all known detection mechanisms and were not envisioned in any adversarial scenario could consistently degrade performance without triggering a response.
5.  **Decision Paralysis from Too Many Scenarios:** If the number of scenarios or policies becomes unmanageably large, or if consensus cannot be reached on scenario relevance or performance metrics, the framework could become too slow or complex to implement effectively.

---

### Competitor 3: **openai/gpt-oss-120b**
## 1. Model‑agnostic Decision Rule  
**Robust Policy‑Selection (RPS)** – choose the policy whose *worst‑case* expected utility, estimated from the data, is highest.  

| Step | Action | Rationale |
|------|--------|-----------|
| 1️⃣| **Define a finite policy set** 𝒫 = {π₁,…,π_K}. | Keeps the problem tractable and interpretable. |
| 2️⃣| **Collect a limited‑size batch** D = { (o_t, a_t, r_t) } where o_t are observable cues, a_t the action taken, r_t the noisy reward. | Works with sparse data; no need for full model. |
| 3️⃣| **Bootstrap‑aggregate utility estimates**: for each π_k, draw B ≤ 200 bootstrap resamples of D, compute the average reward ˆU_k^b on each resample, and retain the **α‑lower‑confidence bound** L_k = 𝑄_{α}( {ˆU_k^b} ), e.g. α = 0.05. | Guarantees a pessimistic estimate that protects against over‑fitting and noise. |
| 4️⃣| **Apply a simple regularizer**: L_k ← L_k – λ·|π_k| (policy complexity, e.g. number of decision rules). | Favors parsimonious policies → interpretability. |
| 5️⃣| **Select** 𝜋* = arg max_{π_k∈𝒫} L_k. | Maximizes the worst‑case plausible performance. |
| 6️⃣| **Provide rationale**: report L_k, bootstrap distribution, and complexity penalty for the chosen π*. | Transparent justification for stakeholders. |

*Implementation notes* – The algorithm needs only reward samples, no model fitting. Bootstrapping is O(K·B·|D|) and can be run on a single CPU.  

---

## 2. Assumptions, Data Requirements & Empirical Test Plan  

| Assumption | What it means | Data needed | Test (hypothetical) |
|------------|---------------|------------|----------------------|
| **A1 – Partial observability** | We only see a noisy proxy o_t of the true state. | Observations o_t, actions a_t, rewards r_t. | Simulate a hidden Markov process; compare estimated L_k with ground‑truth utility. |
| **A2 – Bounded noise** | Reward noise is sub‑Gaussian with known variance proxy σ². | Repeated reward measurements for same (o,a). | Inject Gaussian noise of variance σ² into synthetic rewards; verify coverage of bootstrap L_k (≈95%). |
| **A3 – Limited adversarial contamination** | ≤ ε % of data may be arbitrarily corrupted. | Flagged or suspect records (optional). | Replace ε % of rewards with extreme outliers; check that selected policy remains unchanged if λ is sufficiently large. |
| **A4 – Finite policy set captures near‑optimal actions** | True optimal policy is approximated by some π_k ∈ 𝒫. | Policy library constructed a‑priori (e.g., decision trees of depth ≤ 3). | In simulated environments, generate optimal policy and verify it lies within distance ε of a library member. |

*Empirical plan*: For each regime (A–C) generate 10 synthetic datasets (|D|=200) and run the RPS pipeline. Record (i) selection stability, (ii) coverage of true utility by L_k, and (iii) false‑positive rate of adversarial detection.

---

## 3. Falsifiable Predictions  

| Regime | Prediction (testable) |
|--------|-----------------------|
| **A – Stable DG with modest noise** | *P1*: The selected policy’s true expected reward will be within 5 % of the oracle optimal reward in ≥ 80 % of simulations. |
| **B – Non‑stationarity / concept drift** | *P2*: After a drift point, the bootstrap‑based L_k will drop by > δ (e.g., 0.2) for the previously optimal policy, triggering a policy change in ≤ 5 % of runs (i.e., low false‑alarm). |
| **C – Adversarial manipulation** | *P3*: When ≥ ε = 10 % of rewards are replaced by adversarial values, the chosen policy will remain the same as in the clean data case in ≥ 90 % of runs (robustness). |

If any prediction is violated, the corresponding assumption (A1‑A4) is falsified and the framework must be revised.

---

## 4. Monitoring & Update Protocol  

| Component | Metric / Test | Trigger Condition | Action |
|-----------|---------------|-------------------|--------|
| **Distributional shift** | Kolmogorov‑Smirnov test on o_t windows (size 50) or ADWIN on reward stream. | p‑value < 0.01 **or** drift magnitude > Δ (e.g., 0.15). | Freeze current policy, recompute bootstrap L_k on the most recent window, select new π*. |
| **Data integrity** | % of rewards beyond 3σ of rolling median; checksum anomalies. | > ε_int = 5 % anomalies. | Flag data batch, invoke manual review; if confirmed, discard batch and re‑run RPS on clean data. |
| **Bias detection** | Disparity in L_k across protected groups (e.g., Δ_L > 0.1). | Detected bias persists over 2 consecutive windows. | Apply group‑wise regularization λ_g, re‑evaluate; if still biased, invoke escalation. |
| **Escalation** | Tier‑1: Automated retraining. Tier‑2: Human‑in‑the‑loop review (policy ethics board). Tier‑3: Full rollback to previously validated policy π_{prev}. |
| **Rollback** | If new policy’s L_k < L_{prev} – γ (γ = 0.05) after 30 new samples, revert to π_{prev} and log incident. |

All alerts and actions are logged with timestamps for auditability.

---

## 5. Ethical, Legal & Social Assessment  

| Issue | Stakeholder impact | Mitigation |
|-------|-------------------|------------|
| **Fairness** | Unequal utility across demographic groups. | Include group‑wise penalties in λ·|π|, monitor Δ_L, and enforce a fairness threshold (e.g., 80 % parity). |
| **Transparency** | Decision makers need to understand why a policy was chosen. | Provide the full bootstrap distribution, complexity penalty, and a concise natural‑language explanation for π*. |
| **Privacy** | Observations may contain personal data. | Store only aggregated statistics, apply differential‑privacy noise (ε≈1) to reward counts before bootstrapping. |
| **Liability** | Wrong policy could cause harm (e.g., medical triage). | Keep a versioned audit trail; maintain a “safe fallback” policy vetted by regulators. |
| **Adversarial exploitation** | Attackers could poison data to force a sub‑optimal policy. | Limit ε‑contamination assumption, monitor integrity metrics, and enforce rate‑limiting on data ingestion. |
| **Regulatory compliance** | GDPR, FDA, etc. | Conduct Data Protection Impact Assessment (DPIA), ensure explainability per EU AI Act, and submit the decision rule to the relevant oversight body. |

---

## 6. Justification, Strengths & Limitations  

**Why RPS should work in practice**  
* **Robustness** – By basing the decision on a lower‑confidence bound, we protect against sampling noise, limited data, and modest adversarial contamination.  
* **Model‑agnosticism** – No need to specify a dynamics model; the method works with any observable‑reward data.  
* **Interpretability** – The final rule is a simple policy (e.g., a shallow decision tree) with an explicit utility bound, easy to audit.  
* **Computationally cheap** – Bootstrap + simple penalty scales linearly with the number of policies; feasible on edge devices.  

**Key limitations**  
1. **Policy library dependence** – If the optimal action lies outside 𝒫, performance degrades (violates A4).  
2. **Bootstrap reliability under extreme heavy‑tailed noise** – Sub‑Gaussian assumption may fail, leading to overly optimistic L_k.  
3. **Detection latency** – Drift tests need a few dozen samples; rapid shifts may cause temporary sub‑optimal actions.  
4. **Adversarial budget** – The guarantee holds only up to the assumed ε contamination; a coordinated attack exceeding this budget can break the system.  

**Failure scenarios**  
* Highly non‑stationary environments where the reward distribution changes faster than the monitoring window.  
* When observation space o_t is so severely corrupted that no informative signal remains (complete loss of partial observability).  

---

## Final Decision Rule (Executable Pseudocode)

```python
# Robust Policy Selection (RPS)

import numpy as np
from sklearn.utils import resample

def rps(decision_policies, data, B=150, alpha=0.05, lam=0.01):
    """
    decision_policies : list of callables π_k(o) -> a
    data             : list of tuples (o, a, r)
    B                : number of bootstrap replications
    alpha            : lower‑confidence level (e.g., 0.05)
    lam              : complexity penalty weight
    Returns (π_star, L_star, diagnostics)
    """
    # pre‑compute policy complexities (e.g., number of rules)
    complexities = [policy.complexity() for policy in decision_policies]

    # extract rewards per policy
    rewards_by_policy = {k: [] for k in range(len(decision_policies))}
    for (o, a, r) in data:
        for k, π in enumerate(decision_policies):
            if π(o) == a:                # reward belongs to policy k
                rewards_by_policy[k].append(r)

    # bootstrap lower bounds
    lower_bounds = []
    for k, rewards in rewards_by_policy.items():
        if len(rewards) == 0:
            lower_bounds.append(-np.inf)
            continue
        boot_means = []
        for _ in range(B):
            sample = resample(rewards, replace=True, n_samples=len(rewards))
            boot_means.append(np.mean(sample))
        Lk = np.quantile(boot_means, alpha) - lam * complexities[k]
        lower_bounds.append(Lk)

    # choose best policy
    best_idx = int(np.argmax(lower_bounds))
    π_star = decision_policies[best_idx]
    L_star = lower_bounds[best_idx]

    diagnostics = {
        "lower_bounds": lower_bounds,
        "complexities": complexities,
        "bootstrap_means": boot_means  # last computed (optional)
    }
    return π_star, L_star, diagnostics
```

*Deploy*: run `rps` after every monitoring window (≈50 new samples). If the selected `π_star` differs from the current operational policy, trigger the **Update Protocol** (Section 4).  

---  

*End of answer.*

In [108]:
# — JUDGE PROMPT —
start_time = time.time()

judge_prompt = f"""Question asked:
{question.strip()}

Evaluate these {len(competitors)} responses for accuracy, clarity, reasoning, and overall quality.

Responses:
{together}

Return a clean Markdown response with:
1. Ranked list from best to worst
2. Short 1-sentence explanation for the winner
3. Final winner announcement

Use this exact format:
## Ranking
1. ...
2. ...

## Winner Explanation
...

## Final Winner
**Winner: ...**"""

response = openai.chat.completions.create(
	model="gpt-5",
	messages=[{"role": "user", "content": judge_prompt}],
)

judgement = response.choices[0].message.content
judge_duration = time.time() - start_time

# ── DISPLAY RESULT ──
display(Markdown("# LLM Competition Judgement"))
display(Markdown(f"**Judging time: {judge_duration:.2f} seconds**"))
display(Markdown(judgement))

# ── SAVE TO FILE WITH DURATION ──
with open("final_judgement.md", "w", encoding="utf-8") as f:
	f.write(f"# LLM Competition Judgement\n\n")
	f.write(f"**Judged in {judge_duration:.2f} seconds**\n\n")
	f.write(judgement)

# LLM Competition Judgement

**Judging time: 28.83 seconds**

## Ranking
1. Competitor 1: gpt-5-mini
2. Competitor 2: gemini-2.5-flash
3. Competitor 3: openai/gpt-oss-120b

## Winner Explanation
Competitor 1 provides the most accurate and implementable framework by combining robust off-policy evaluation (DR/IPS with clipping), conservative selection (LCB/CVaR), explicit perturbation modeling, clear monitoring/rollback protocols, and thorough ELSI considerations.

## Final Winner
**Winner: gpt-5-mini**